# Chapter 8: Continuing Expressions

In [1]:
import polars as pl
pl.show_versions()  # The book is built with Polars version 1.0.0

In [2]:
import math
import numpy as np

print(f"{math.pi=}")
rng = np.random.default_rng(1729)
print(f"{rng.random()=}")

## Types of Operations

### Example A: Element-Wise Operations

In [5]:
penguins = (
    pl.read_csv("data/penguins.csv", null_values="NA")
    .select(
        "species",
        "island",
        "sex",
        "year",
        pl.col("body_mass_g").alias("mass") / 1000
    ))
penguins.with_columns(
    pl.col("mass").sqrt().alias("mass_sqrt"),  
    pl.col("mass").interpolate().alias("mass_filled")  
)

### Example B: Operations that Summarize to One

In [7]:
penguins.select(
    pl.col("mass").mean(),
    pl.col("island").mode().first()  
)

### Example C: Operations that Summarize to One or More

In [9]:
penguins.select(
    pl.col("island").unique()
)

### Example D: Operations that Extend

In [11]:
penguins.select(
    pl.col("species")
    .unique()  
    .repeat_by(3000)  
    .explode()  
    .extend_constant("Saiyan", n=1)  
)

## Element-Wise Operations

### Operations That Perform Mathematical Transformations

In [14]:
(
    pl.DataFrame({"x": [-2, 0, 0.5, 1, math.e, 1000]})
    .with_columns(
        abs=pl.col("x").abs(),
        exp=pl.col("x").exp(),
        log2=pl.col("x").log(2),  
        log10=pl.col("x").log10(),
        log1p=pl.col("x").log1p(),
        sign=pl.col("x").sign(),
        sqrt=pl.col("x").sqrt(),
    )
)

### Operations Related to Trigonometry

In [16]:
(
    pl.DataFrame({"x": [-math.pi, 0, 1, math.pi, 2*math.pi, 90, 180, 360]})
    .with_columns(
        arccos=pl.col("x").arccos(),  
        cos=pl.col("x").cos(),
        degrees=pl.col("x").degrees(),
        radians=pl.col("x").radians(),
        sin=pl.col("x").sin(),
    )
)

### Operations That Round and Categorize

In [18]:
(
    pl.DataFrame({"x": [-6, -0.5, 0, 0.5, math.pi, 9.9, 9.99, 9.999]})
    .with_columns(
        ceil=pl.col("x").ceil(),
        clip=pl.col("x").clip(-1, 1),
        cut=pl.col("x").cut([-1, 1], labels=["bad", "neutral", "good"]),  
        floor=pl.col("x").floor(),
        qcut=pl.col("x").qcut([0.5], labels=["below median", "above median"]),
        round2=pl.col("x").round(2),
        round0=pl.col("x").round(0),  
    )
)

### Operations for Missing or Infinite Values

In [20]:
x = [42, math.nan, None, math.inf, -math.inf]
(
    pl.DataFrame({"x": x})
    .with_columns(
        fill_nan=pl.col("x").fill_nan(999),
        fill_null=pl.col("x").fill_null(0),
        is_finite=pl.col("x").is_finite(),
        is_infinite=pl.col("x").is_finite(),
        is_nan=pl.col("x").is_nan(),
        is_null=pl.col("x").is_null(),
    )
)

In [21]:
(
    pl.DataFrame({"x": x})
    .with_columns(
        fill_both=pl.col("x").fill_nan(0).fill_null(0),
        is_either=(
            pl.col("x").is_nan() | pl.col("x").is_null()
        ),
    )
)

### Other Operations

In [23]:
(
    pl.DataFrame({"x": ["here", "there", "their", "they're"]})
    .with_columns(
        hash=pl.col("x").hash(seed=1337),  
        repeat_by=pl.col("x").repeat_by(3),
        replace=pl.col("x").replace({
            "here": "there",
            "they're": "they are",
        }),
    )
)

## Nonreducing Series-Wise Operations

### Operations That Accumulate

In [26]:
(
    pl.DataFrame({"x": [0, 1, 2, None, 2, np.NaN, -1, 2]})
    .with_columns(
        cum_count=pl.col("x").cum_count(),  
        cum_max=pl.col("x").cum_max(),
        cum_min=pl.col("x").cum_min(),
        cum_prod=pl.col("x").cum_prod(reverse=True),  
        cum_sum=pl.col("x").cum_sum(),
        diff=pl.col("x").diff(),
        pct_change=pl.col("x").pct_change(),
    )
)

### Operations That Fill and Shift

In [28]:
(
    pl.DataFrame({"x": [-1, 0, 1, None, None, 3, 4, math.nan, 6]})
    .with_columns(
        backward_fill=pl.col("x").backward_fill(),  
        forward_fill=pl.col("x").forward_fill(limit=1),
        interp1=pl.col("x").interpolate(method="linear"),  
        interp2=pl.col("x").interpolate(method="nearest"),
        shift1=pl.col("x").shift(1),
        shift2=pl.col("x").shift(-2),
    )
)

### Operations Related to Duplicate Values

In [30]:
(
    pl.DataFrame({"x": ["A", "C", "D", "C"]})  
    .with_columns(
        is_duplicated=pl.col("x").is_duplicated(),
        is_first_distinct=pl.col("x").is_first_distinct(),
        is_last_distinct=pl.col("x").is_last_distinct(),
        is_unique=pl.col("x").is_unique(),
    )
)

### Operations That Compute Rolling Statistics

In [32]:
stock = (
    pl.read_csv("data/stock/nvda/2023.csv", try_parse_dates=True)
    .select("date", "close")
    .with_columns(
        ewm_mean=pl.col("close").ewm_mean(com=7, ignore_nulls=True),  
        rolling_mean=pl.col("close").rolling_mean(window_size=7),
        rolling_min=pl.col("close").rolling_min(window_size=7),
    )
)
stock

In [33]:
from matplotlib.dates import DateFormatter
stock.plot.line(
    x="date",
    y=["close", "ewm_mean", "rolling_mean", "rolling_min"],
    xformatter=DateFormatter("%b %Y")
)

### Operations That Sort

In [35]:
(
    pl.DataFrame({
        "x": [1, 3, None, 3, 7],
        "y": ["D", "I", "S", "C", "O"],
    })
    .with_columns(
        arg_sort=pl.col("x").arg_sort(),
        shuffle=pl.col("x").shuffle(seed=7),
        sort=pl.col("x").sort(nulls_last=True),
        sort_by=pl.col("x").sort_by("y"),
        reverse=pl.col("x").reverse(),
        rank=pl.col("x").rank(),
    )
)

### Other Operations

In [37]:
(
    pl.DataFrame({"x": [33, 33, 27, 33, 60, 60, 60, 33, 60]})
    .with_columns(
        rle_id=pl.col("x").rle_id(),
    )
)

## Series-Wise Operations that Summarize to One

In [39]:
(
    pl.DataFrame({"x": [1, 3, 3, 7]})
    .with_columns(
        mean=pl.col("x").mean(),
    )
)

In [40]:
(
    pl.DataFrame({
        "cluster": ["a", "a", "b", "b"],
        "x": [1, 3, 3, 7]
    })
    .group_by("cluster")
    .agg(
        mean=pl.col("x").mean(),
    )
)

### Operations That Are Quantifiers

In [42]:
(
    pl.DataFrame({
        "x": [True, False, False],
        "y": [True, True, True],
        "z": [False, False, False],
        })
    .select(
        pl.all().all().name.suffix("_all"),
        pl.all().any().name.suffix("_any"),
    )
)

### Operations That Compute Statistics

In [44]:
samples = rng.normal(loc=5, scale=3, size=1_000_000)

(
    pl.DataFrame({"x": samples})
    .select(
        max=pl.col("x").max(),
        mean=pl.col("x").mean(),
        quantile=pl.col("x").quantile(quantile=0.95),
        skew=pl.col("x").skew(),
        std=pl.col("x").std(),
        sum=pl.col("x").sum(),
        var=pl.col("x").var(),
    )
)

### Operations That Count

In [46]:
samples = pl.Series(rng.integers(low=0, high=10_000, size=1_729))
samples[403] = None  
df_ints = (
    pl.DataFrame({"x": samples})
    .with_row_index()  
)
df_ints.slice(400, 6)  

In [47]:
df_ints.select(
    approx_n_unique=pl.col("x").approx_n_unique(),
    count=pl.col("x").count(),
    len=pl.col("x").len(),
    n_unique=pl.col("x").n_unique(),
    null_count=pl.col("x").null_count(),
)

### Other Operations

In [49]:
df_ints.select(
    arg_min=pl.col("x").arg_min(),
    first=pl.col("x").first(),
    get=pl.col("x").get(403),  
    implode=pl.col("x").implode(),
    last=pl.col("x").last(),
    upper_bound=pl.col("x").upper_bound(),
)

## Series-Wise Operations that Summarize to One or More

### Operations Related to Unique Values

In [52]:
(
    pl.DataFrame({"x": ["A", "C", "D", "C"]})
    .select(
        arg_unique=pl.col("x").arg_unique(),
        unique=pl.col("x").unique(maintain_order=True),  
        unique_counts=pl.col("x").unique_counts(),
        value_counts=pl.col("x").value_counts(),  
    )
)

### Operations That Select

In [54]:
df_ints.select(
    bottom_k=pl.col("x").bottom_k(7),  
    head=pl.col("x").head(7),
    sample=pl.col("x").sample(7),
    slice=pl.col("x").slice(400, 7),
    gather=pl.col("x").gather([1, 1, 2, 3, 5, 8, 13]),
    gather_every=pl.col("x").gather_every(247),  
    top_k=pl.col("x").top_k(7),
)

### Operations That Drop Missing Values

In [56]:
x = [None, 1, 2, 3, np.NaN]
(
    pl.DataFrame({"x": x})
    .select(
        drop_nans=pl.col("x").drop_nans(),
        drop_nulls=pl.col("x").drop_nulls()
    )
)

### Other Operations

In [58]:
numbers = [33, 33, 27, 33, 60, 60, 60, 33, 60]

(
    pl.DataFrame({"x": numbers})
    .select(
        arg_true=(pl.col("x") >= 60).arg_true(),  
    )
)

In [59]:
(
    pl.DataFrame({"x": numbers})
    .select(
        mode=pl.col("x").mode(),
    )
)

In [60]:
(
    pl.DataFrame({"x": numbers})
    .select(
        reshape=pl.col("x").reshape((3, 3)),  
    )
)

In [61]:
(
    pl.DataFrame({"x": numbers})
    .select(
        rle=pl.col("x").rle(),  
    )
)

In [62]:
(
    pl.DataFrame({"x": numbers})
    .select(
        rle=pl.col("x").sort().search_sorted(42),  
    )
)

## Series-Wise Operations that Extend

In [64]:
(
    pl.DataFrame({
        "x": [["a", "b"], ["c", "d"]],
    })
    .select(
        explode=pl.col("x").explode()
    )
)

## Conclusion